# <span style="color: blue;">Итераторы</span>

### Цикл for и модуль dis

Напоминание: оператор `for` в `Python` работает с любой последовательностью

In [1]:
import dis
dis.dis("for x in xs: do_something(name)")

  1           0 SETUP_LOOP              20 (to 22)
              2 LOAD_NAME                0 (xs)
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_NAME               1 (x)
             10 LOAD_NAME                2 (do_something)
             12 LOAD_NAME                3 (name)
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK
        >>   22 LOAD_CONST               0 (None)
             24 RETURN_VALUE


Инструкция `GET_ITER` вызывает у аргумента оператора `for` метод `__iter__`, который возвращает итератор.

Инструкция `FOR_ITER` вызывает метод `__next__` у итератора до тех пор, пока не будет поднято исключение `StopIteration`.

### Протокол итераторов

Протокол итераторов состоит из двух методов:
* Метод `__iter__` возвращает экземпляр класса, реализующего протокол итераторов (например, `self`)
* Метод `__next__` возвращает следующий по порядку элемент итератора<br/>
Если такого элемента нет, то метод должен поднять исключение `StopIteration`

Важный инвариант метода `__next__`:<br/>
если метод поднял исключение `StopIteration`, то все последующие вызовы `__next__` тоже должны поднимать исключение.

Если реализовывать оба метода в одном классе, то _`iterator`_ также является и _`iterable`_.

### Коллекции и итераторы

Для коллекций обычно нет смысла реализовывать протокол итераторов целиком, достаточно реализовать только метод `__iter__`.

Иногда элементы коллекции можно перечислить более чем одним способом. В этом случае удобно реализовывать дополнительные методы, возвращающие итераторы:

In [ ]:
class BinaryTree:
    def __iter__(self):
        return self.inorder_iter()
    
    def preorder_iter(self):
        # ...
        
    def inorder_iter(self):
        return InOrderIterator(self)
    
    def postorder_iter(self):
        # ...

### Функции iter и next

У функции **`iter`** две формы вызова:
* принимает _`iterable`_ и вызывает у него метод `__iter__`
* принимает функцию и терминальное значение и вызывает функцию до тех пор, пока она не вернёт нужное значение:

In [ ]:
from functools import partial

with open(path, "rb") as handle:
    read_block = partial(handle.read, 64)
    
#     while read_block() != "":
#         pass
    
    for block in iter(read_block, ""):
        do_something(block)

Ещё интересные примеры: http://bit.ly/beautiful-python

Функция **`next`** принимает _`iterator`_ и вызывает у него метод `__next__`. 

Можно также указать значение, которое нужно вернуть в случае возникновения исключения `StopIteration`:

In [3]:
next(iter([1, 2, 3]))

1

In [5]:
it = iter([1, 2, 3])
next(it)

1

In [6]:
next(it)

2

In [7]:
next(it) 

3

In [8]:
next(it)

StopIteration: 

In [9]:
next(iter([]), 42)

42

### “Семантика” оператора for

Напоминание:

In [ ]:
for x in xs:
    do_something(x)

Процесс исполнения оператора `for` можно концептуально записать так:

In [ ]:
it = iter(xs)
while True:
    try:
        x = next(it)
    except StopIteration:
        break
    do_something(x)

### Протокол итераторов и операторы in и not in

Операторы `in` и `not in` используют “магический” метод `__contains__`, который возвращает `True`, если переданный элемент содержится в экземпляре класса.

По умолчанию метод `__contains__` реализован через протокол итераторов _(концептуально можно записать так)_:

In [ ]:
class object:
    # ...
    def __contains__(self, target):
        for item in self:
            if item == target:
                return True
        return False

Пример:

In [ ]:
id = Identity()
5 in id # ≡ id.__contains__(5)

In [ ]:
42 not in id # ≡ not id.__contains__(42)

Как правило, этот метод перекрывается. Например, для `set` или `range` и т.п.

Но зато понятно, как `in` работает, например, для `map`.

### Протокол итераторов и реализация “по умолчанию”

В Python предусмотрен упрощённый вариант реализации протокола итераторов с использованием метода `__getitem__`.

Метод `__getitem__` принимает один аргумент — индекс элемента в последовательности и:
* либо возвращает элемент, соответствующий индексу
* либо поднимает `IndexError`, если элемента с таким индексом нет

Пример:

In [9]:
class Identity:
    def __getitem__(self, idx):
        if idx > 5:
            raise IndexError(idx)
        return idx

list(Identity())

[0, 1, 2, 3, 4, 5]

In [10]:
it = iter(Identity())
next(it)

0

In [11]:
next(it)

1

То есть при реализации `__getitem__` у нас автоматически появляется `__iter__` вместе с _возможно_ неожиданной реализацией по умолчанию.

### “Семантика” упрощённого протокола итераторов

In [ ]:
class seq_iter:
    def __init__(self, instance):
        self.instance = instance
        self.idx = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            res = self.instance[self.idx]
        except IndexError:
            raise StopIteration
            
        self.idx += 1
        return res

Концептуально `object` можно записать так:

In [ ]:
class object:
    # ...
    
    def __iter__(self):
        if not hasattr(self, "__getitem__"):
            cls = self.__class__
            msg = "{} object is not iterable"
            raise TypeError(msg.format(cls.__name__))
        return seq_iter(self)

### Итератор можно исчерпать

In [10]:
it = iter([1, 2, 3])
for x in it:
    print(x)
list(it)

1
2
3


[]

### Резюме: итераторы

В `Python iterator` иногда является и `iterable`.

Итератор (_`iterator`_) — это экземпляр класса, который реализует `__next__`.

_`iterable`_ — реализует `__iter__` для получения итератора.

Если класс содержит оба эти метода, то он является и _`iterable`_, и _`iterator`_.

Альтернативно можно воспользоваться реализацией этих методов по умолчанию и определить метод `__getitem__`.

Протокол итераторов используется:
* оператором `for`
* операторами `in` и `not in`

Протокол итераторов реализуется всеми встроенными коллекциями, а также, например, файлами и объектами типа `map, filter и zip`.
